In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('./data.csv', encoding='latin')

### 数据预览

In [7]:
df.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,¿¨ºÅ1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,¿¨ºÅ1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,¿¨ºÅ1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,¿¨ºÅ1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,¿¨ºÅ1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


### 数据维度 4754行，90列

In [8]:
df.shape

(4754, 90)

### 查看列的数据类型分布

In [9]:
df.dtypes.value_counts()

float64    70
int64      13
object      7
dtype: int64

In [12]:
df.select_dtypes(include='object').columns.values

array(['trade_no', 'bank_card_no', 'reg_preference_for_trad', 'source',
       'id_name', 'latest_query_time', 'loans_latest_time'], dtype=object)

In [13]:
df.select_dtypes(include='int64').columns.values

array(['Unnamed: 0', 'custid', 'take_amount_in_later_12_month_highest',
       'repayment_capability', 'is_high_user', 'historical_trans_amount',
       'trans_amount_3_month', 'abs', 'avg_price_last_12_month',
       'max_cumulative_consume_later_1_month',
       'pawns_auctions_trusts_consume_last_1_month',
       'pawns_auctions_trusts_consume_last_6_month', 'status'],
      dtype=object)

In [14]:
df.select_dtypes(include='float64').columns.values

array(['low_volume_percent', 'middle_volume_percent',
       'trans_amount_increase_rate_lately', 'trans_activity_month',
       'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'regional_mobility', 'student_feature',
       'number_of_trans_from_2011', 'first_transaction_time',
       'historical_trans_day', 'rank_trad_1_month',
       'avg_consume_less_12_valid_month', 'top_trans_count_last_1_month',
       'avg_price_top_last_12_valid_month', 'trans_top_time_last_1_month',
       'trans_top_time_last_6_month', 'consume_top_time_last_1_month',
       'consume_top_time_last_6_month',
       'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_month',
       'trans_fail_top_count_enum_last_6_month',
       'trans_fail_top_count_enum_last_12_month',
       'consume_mini_time_last_1_month',
       'max_consume_count_later_6_month',
       'railway_consume_count_last_12_month',
       'jewelry_consume_count_last_6_mont

In [15]:
df['first_transaction_time'].head(5)

0    20130817.0
1    20160402.0
2    20170617.0
3    20130516.0
4    20170312.0
Name: first_transaction_time, dtype: float64

### first_transaction_time被pandas识别为float64，后期需要做数据类型转换

### 查看status值的分布情况

In [16]:
df['status'].value_counts()

0    3561
1    1193
Name: status, dtype: int64

### 删除无关列
#### 怎么确定是和status无关的列呢？是不是要计算相关性才知道呢？这块不知道怎么弄

In [18]:
df = df.drop(['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no'], axis=1)

### 缺失数据处理，主要使用删除法、填补法和插值法
#### 删除法
##### 当数据中的某个变量大部分值都是缺失值，可以考虑删除改变量；当缺失值是随机分布的，且缺失的数量并不是很多是，也可以删除这些缺失的观测。
##### 替补法
##### 对于连续型变量，如果变量的分布近似或就是正态分布的话，可以用均值替代那些缺失值；如果变量是有偏的，可以使用中位数来代替那些缺失值；对于离散型变量，我们一般用众数去替换那些存在缺失的观测。
#### 插补法
##### 插补法是基于蒙特卡洛模拟法，结合线性模型、广义线性模型、决策树等方法计算出来的预测值替换缺失值。

In [20]:
df.isnull().sum()

low_volume_percent                          2
middle_volume_percent                       2
take_amount_in_later_12_month_highest       0
trans_amount_increase_rate_lately           3
trans_activity_month                        2
trans_activity_day                          2
transd_mcc                                  2
trans_days_interval_filter                  8
trans_days_interval                         2
regional_mobility                           2
student_feature                          2998
repayment_capability                        0
is_high_user                                0
number_of_trans_from_2011                   2
first_transaction_time                      2
historical_trans_amount                     0
historical_trans_day                        2
rank_trad_1_month                           2
trans_amount_3_month                        0
avg_consume_less_12_valid_month             2
abs                                         0
top_trans_count_last_1_month      

#### 总记录行数为4754行，如果大多数都缺失，则删除该列，否则使用平均值进行填充

In [21]:
df = df.drop(['student_feature'], axis=1)

### 划分数据集

In [24]:
train_x, train_y = train_test_split(df, test_size=0.3, random_state=2018)